In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import warnings
from datetime import datetime, timedelta
import zipfile
import pickle

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

In [ ]:
df = pd.read_csv('datasets/tsunami_dataset.csv')

In [ ]:
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_info = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percentage
}).sort_values('Missing Percentage', ascending=False)

categorical_cols = ['EVENT_VALIDITY', 'CAUSE', 'REGION']
for col in categorical_cols:
    if col in df.columns:
        df[col].value_counts().head(10)

if 'TS_INTENSITY' in df.columns:
    ts_intensity_counts = df['TS_INTENSITY'].value_counts()

In [ ]:
df_clean = df.copy()
df_clean = df_clean.dropna(how='all')
df_clean = df_clean.dropna(subset=['EQ_MAGNITUDE'])
df_clean = df_clean[(df_clean['EQ_MAGNITUDE'] >= 0) & (df_clean['EQ_MAGNITUDE'] <= 10)]

if 'EQ_DEPTH' in df_clean.columns:
    median_depth = df_clean['EQ_DEPTH'].median()
    df_clean['EQ_DEPTH'] = df_clean['EQ_DEPTH'].fillna(median_depth)
    df_clean = df_clean[(df_clean['EQ_DEPTH'] >= 0) & (df_clean['EQ_DEPTH'] <= 800)]

if 'LATITUDE' in df_clean.columns and 'LONGITUDE' in df_clean.columns:
    df_clean = df_clean[
        (df_clean['LATITUDE'].between(-90, 90)) & 
        (df_clean['LONGITUDE'].between(-180, 180))
    ]

df_clean['TSUNAMI_OCCURRED'] = 0
if 'TS_INTENSITY' in df_clean.columns:
    df_clean.loc[df_clean['TS_INTENSITY'].notna() & (df_clean['TS_INTENSITY'] > 0), 'TSUNAMI_OCCURRED'] = 1

if 'EVENT_VALIDITY' in df_clean.columns:
    tsunami_indicators = ['Definite Tsunami', 'Probable Tsunami']
    mask = df_clean['EVENT_VALIDITY'].isin(tsunami_indicators)
    df_clean.loc[mask, 'TSUNAMI_OCCURRED'] = 1

if 'YEAR' in df_clean.columns:
    df_clean = df_clean[(df_clean['YEAR'] >= 1800) | df_clean['YEAR'].isna()]

if 'CAUSE' in df_clean.columns:
    df_clean['CAUSE'] = df_clean['CAUSE'].fillna('Unknown')

if 'REGION' in df_clean.columns:
    df_clean['REGION'] = df_clean['REGION'].fillna('Unknown')

In [ ]:
features_df = df_clean.copy()

features_df['MAG_CATEGORY'] = pd.cut(
    features_df['EQ_MAGNITUDE'], 
    bins=[0, 4.5, 6.0, 7.0, 8.0, 10.0],
    labels=['Weak', 'Moderate', 'Strong', 'Major', 'Great']
)

features_df['MAG_SQUARED'] = features_df['EQ_MAGNITUDE'] ** 2
features_df['IS_MAJOR_EQ'] = (features_df['EQ_MAGNITUDE'] >= 7.0).astype(int)

if 'EQ_DEPTH' in features_df.columns:
    features_df['DEPTH_CATEGORY'] = pd.cut(
        features_df['EQ_DEPTH'],
        bins=[0, 70, 300, 700, 800],
        labels=['Shallow', 'Intermediate', 'Deep', 'Very_Deep']
    )
    
    features_df['IS_SHALLOW'] = (features_df['EQ_DEPTH'] <= 70).astype(int)
    features_df['DEPTH_MAG_RATIO'] = features_df['EQ_DEPTH'] / features_df['EQ_MAGNITUDE']

def assign_risk_zone(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return 'Unknown'
    
    if (lat >= 35 and lat <= 45 and lon >= 135 and lon <= 145) or \
       (lat >= -50 and lat <= -30 and lon >= -80 and lon <= -60) or \
       (lat >= 30 and lat <= 50 and lon >= -130 and lon <= -110):
        return 'Very_High'
    
    elif (lon >= -180 and lon <= -100) or (lon >= 100 and lon <= 180):
        if abs(lat) <= 60:
            return 'High'
    
    elif (lon >= -100 and lon <= 20):
        return 'Moderate'
    
    elif (lat >= 30 and lat <= 45 and lon >= 0 and lon <= 40):
        return 'Low_Moderate'
    
    return 'Low'

if 'LATITUDE' in features_df.columns and 'LONGITUDE' in features_df.columns:
    features_df['RISK_ZONE'] = features_df.apply(
        lambda row: assign_risk_zone(row['LATITUDE'], row['LONGITUDE']), axis=1
    )

def is_oceanic(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return 0
    
    if lat >= 25 and lat <= 70 and lon >= -160 and lon <= -50:
        return 0
    elif lat >= 35 and lat <= 75 and lon >= -10 and lon <= 180:
        return 0
    elif lat >= -35 and lat <= 35 and lon >= -20 and lon <= 55:
        return 0
    elif lat >= -45 and lat <= -10 and lon >= 110 and lon <= 155:
        return 0
    elif lat >= -55 and lat <= 15 and lon >= -85 and lon <= -30:
        return 0
    
    return 1

if 'LATITUDE' in features_df.columns and 'LONGITUDE' in features_df.columns:
    features_df['IS_OCEANIC'] = features_df.apply(
        lambda row: is_oceanic(row['LATITUDE'], row['LONGITUDE']), axis=1
    )

if 'YEAR' in features_df.columns:
    features_df['DECADE'] = (features_df['YEAR'] // 10) * 10
    features_df['IS_RECENT'] = (features_df['YEAR'] >= 1950).astype(int)

if 'MONTH' in features_df.columns:
    features_df['SEASON'] = features_df['MONTH'].apply(
        lambda x: 'Winter' if x in [12, 1, 2] else
                  'Spring' if x in [3, 4, 5] else
                  'Summer' if x in [6, 7, 8] else
                  'Fall' if x in [9, 10, 11] else 'Unknown'
    )

label_encoders = {}
categorical_features = ['RISK_ZONE', 'MAG_CATEGORY', 'DEPTH_CATEGORY', 'CAUSE', 'REGION']

for feature in categorical_features:
    if feature in features_df.columns:
        le = LabelEncoder()
        feature_series = features_df[feature].astype(str).fillna('Unknown')
        if hasattr(feature_series, 'cat'):
            feature_series = feature_series.cat.add_categories('Unknown').fillna('Unknown')
        features_df[f'{feature}_ENCODED'] = le.fit_transform(feature_series)
        label_encoders[feature] = le

if 'TSUNAMI_OCCURRED' in features_df.columns:
    numeric_features = features_df.select_dtypes(include=[np.number]).columns
    correlations = features_df[numeric_features].corr()['TSUNAMI_OCCURRED'].sort_values(key=abs, ascending=False)

In [ ]:
feature_columns = [
    'EQ_MAGNITUDE', 'EQ_DEPTH', 'LATITUDE', 'LONGITUDE',
    'MAG_SQUARED', 'IS_MAJOR_EQ', 'IS_SHALLOW', 'IS_OCEANIC',
    'RISK_ZONE_ENCODED', 'MAG_CATEGORY_ENCODED', 'DEPTH_CATEGORY_ENCODED'
]

available_features = []
for col in feature_columns:
    if col in features_df.columns:
        available_features.append(col)

ml_data = features_df.dropna(subset=['TSUNAMI_OCCURRED']).copy()

X = ml_data[available_features].copy()
y = ml_data['TSUNAMI_OCCURRED'].copy()

for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(0)

scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_minmax = minmax_scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_minmax_df = pd.DataFrame(X_minmax, columns=X.columns, index=X.index)

feature_stats = pd.DataFrame({
    'Feature': X.columns,
    'Mean': X.mean(),
    'Std': X.std(),
    'Min': X.min(),
    'Max': X.max(),
    'Missing_Count': X.isnull().sum()
})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

split_data = {
    'X_train': X_train_split,
    'X_val': X_val,
    'X_test': X_test,
    'y_train': y_train_split,
    'y_val': y_val,
    'y_test': y_test,
    'feature_names': list(X.columns),
    'scaler': scaler
}

In [ ]:
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        class_weight='balanced'
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        scale_pos_weight=len(y_train_split[y_train_split==0])/len(y_train_split[y_train_split==1])
    ),
    'Support Vector Machine': SVC(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        random_state=42,
        class_weight='balanced',
        probability=True
    ),
    'Logistic Regression': LogisticRegression(
        C=1.0,
        random_state=42,
        class_weight='balanced',
        max_iter=1000
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
}

trained_models = {}
model_scores = {}

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train_split, y_train_split)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None
    
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    cv_scores = cross_val_score(model, X_train_split, y_train_split, cv=5, scoring='f1')
    
    trained_models[name] = model
    model_scores[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'training_time': training_time
    }

results_df = pd.DataFrame(model_scores).T
results_df = results_df.round(3)

best_model_name = results_df['f1'].idxmax()
best_model = trained_models[best_model_name]

if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train_split.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)

In [ ]:
y_test_pred = best_model.predict(X_test)
y_test_pred_proba = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, 'predict_proba') else None

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

cm = confusion_matrix(y_test, y_test_pred)

performance_summary = {
    'Model': best_model_name,
    'Test_Accuracy': test_accuracy,
    'Test_Precision': test_precision,
    'Test_Recall': test_recall,
    'Test_F1': test_f1,
    'True_Negatives': cm[0,0],
    'False_Positives': cm[0,1],
    'False_Negatives': cm[1,0],
    'True_Positives': cm[1,1]
}

In [ ]:
USGS_ENDPOINTS = {
    'past_hour_m45': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_hour.geojson',
    'past_day_m45': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_day.geojson',
    'past_hour_m25': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.geojson',
    'past_day_all': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'
}

def fetch_earthquake_data(endpoint_key='past_hour_m45', timeout=10):
    try:
        url = USGS_ENDPOINTS[endpoint_key]
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        data = response.json()
        return data
    except:
        return None

def parse_earthquake_features(earthquake_data):
    if not earthquake_data or 'features' not in earthquake_data:
        return pd.DataFrame()
    
    earthquakes = []
    
    for feature in earthquake_data['features']:
        props = feature['properties']
        coords = feature['geometry']['coordinates']
        
        earthquake = {
            'id': feature['id'],
            'magnitude': props.get('mag'),
            'place': props.get('place', ''),
            'time': pd.to_datetime(props.get('time'), unit='ms'),
            'longitude': coords[0] if len(coords) > 0 else None,
            'latitude': coords[1] if len(coords) > 1 else None,
            'depth': coords[2] if len(coords) > 2 else None,
            'magType': props.get('magType', ''),
            'nst': props.get('nst'),
            'gap': props.get('gap'),
            'dmin': props.get('dmin'),
            'rms': props.get('rms'),
            'net': props.get('net', ''),
            'updated': pd.to_datetime(props.get('updated'), unit='ms') if props.get('updated') else None,
            'type': props.get('type', ''),
            'status': props.get('status', ''),
            'tsunami': props.get('tsunami', 0),
            'url': props.get('url', '')
        }
        earthquakes.append(earthquake)
    
    df = pd.DataFrame(earthquakes)
    return df

test_data = fetch_earthquake_data('past_hour_m45')

if test_data:
    test_df = parse_earthquake_features(test_data)

def get_earthquake_monitoring_status():
    status = {
        'timestamp': datetime.now(),
        'api_status': 'unknown',
        'last_earthquake_count': 0,
        'connection_test': False
    }
    
    test_data = fetch_earthquake_data('past_hour_m25')
    if test_data:
        status['api_status'] = 'connected'
        status['last_earthquake_count'] = len(test_data.get('features', []))
        status['connection_test'] = True
    else:
        status['api_status'] = 'disconnected'
    
    return status

monitoring_status = get_earthquake_monitoring_status()

In [ ]:
def filter_tsunami_threats(earthquake_df, min_magnitude=6.5, max_depth=70, oceanic_only=True):
    if earthquake_df.empty:
        return earthquake_df
    
    threats = earthquake_df.copy()
    threats = threats[threats['magnitude'] >= min_magnitude]
    threats = threats[threats['depth'] <= max_depth]
    
    if oceanic_only:
        threats['is_oceanic'] = threats.apply(
            lambda row: is_oceanic(row['latitude'], row['longitude']), axis=1
        )
        threats = threats[threats['is_oceanic'] == 1]
    
    return threats

def prepare_features_for_prediction(earthquake_df):
    if earthquake_df.empty:
        return pd.DataFrame()
    
    features = earthquake_df.copy()
    
    feature_mapping = {
        'magnitude': 'EQ_MAGNITUDE',
        'depth': 'EQ_DEPTH',
        'latitude': 'LATITUDE',
        'longitude': 'LONGITUDE'
    }
    
    features = features.rename(columns=feature_mapping)
    
    features['MAG_SQUARED'] = features['EQ_MAGNITUDE'] ** 2
    features['IS_MAJOR_EQ'] = (features['EQ_MAGNITUDE'] >= 7.0).astype(int)
    features['IS_SHALLOW'] = (features['EQ_DEPTH'] <= 70).astype(int)
    
    features['RISK_ZONE'] = features.apply(
        lambda row: assign_risk_zone(row['LATITUDE'], row['LONGITUDE']), axis=1
    )
    
    features['IS_OCEANIC'] = features.apply(
        lambda row: is_oceanic(row['LATITUDE'], row['LONGITUDE']), axis=1
    )
    
    features['MAG_CATEGORY'] = pd.cut(
        features['EQ_MAGNITUDE'], 
        bins=[0, 4.5, 6.0, 7.0, 8.0, 10.0],
        labels=['Weak', 'Moderate', 'Strong', 'Major', 'Great']
    )
    
    features['DEPTH_CATEGORY'] = pd.cut(
        features['EQ_DEPTH'],
        bins=[0, 70, 300, 700, 800],
        labels=['Shallow', 'Intermediate', 'Deep', 'Very_Deep']
    )
    
    for feature_name, encoder in label_encoders.items():
        if feature_name.replace('_ENCODED', '') in features.columns:
            original_col = feature_name.replace('_ENCODED', '')
            try:
                feature_series = features[original_col].astype(str).fillna('Unknown')
                known_mask = feature_series.isin(encoder.classes_)
                if not known_mask.all():
                    most_frequent_class = encoder.classes_[0] if len(encoder.classes_) > 0 else 'Unknown'
                    feature_series.loc[~known_mask] = most_frequent_class
                
                features[feature_name] = encoder.transform(feature_series)
            except:
                if original_col in features.columns:
                    unique_vals = features[original_col].unique()
                    mapping = {val: i for i, val in enumerate(unique_vals)}
                    features[feature_name] = features[original_col].map(mapping).fillna(0)
                else:
                    features[feature_name] = 0
    
    return features

def assess_tsunami_risk(earthquake_df, model, scaler, feature_columns):
    if earthquake_df.empty:
        return pd.DataFrame()
    
    features = prepare_features_for_prediction(earthquake_df)
    
    available_columns = [col for col in feature_columns if col in features.columns]
    missing_columns = [col for col in feature_columns if col not in features.columns]
    
    X_pred = features[available_columns].copy()
    
    for col in feature_columns:
        if col not in X_pred.columns:
            X_pred[col] = 0
    
    X_pred = X_pred.fillna(0)
    
    X_pred_scaled = scaler.transform(X_pred)
    
    predictions = model.predict(X_pred_scaled)
    probabilities = model.predict_proba(X_pred_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    results = earthquake_df.copy()
    results['tsunami_prediction'] = predictions
    results['tsunami_probability'] = probabilities if probabilities is not None else predictions
    results['risk_level'] = 'Low'
    
    if probabilities is not None:
        results.loc[results['tsunami_probability'] >= 0.7, 'risk_level'] = 'High'
        results.loc[(results['tsunami_probability'] >= 0.3) & (results['tsunami_probability'] < 0.7), 'risk_level'] = 'Medium'
    else:
        results.loc[results['tsunami_prediction'] == 1, 'risk_level'] = 'High'
    
    return results

sample_earthquakes = pd.DataFrame({
    'id': ['test1', 'test2', 'test3'],
    'magnitude': [7.2, 6.8, 5.5],
    'latitude': [38.0, -15.0, 35.0],
    'longitude': [142.0, -175.0, 25.0],
    'depth': [30.0, 15.0, 80.0],
    'place': ['Japan', 'Pacific Ocean', 'Mediterranean'],
    'time': [datetime.now()] * 3
})

threats = filter_tsunami_threats(sample_earthquakes)

if len(threats) > 0:
    risk_assessment = assess_tsunami_risk(threats, best_model, scaler, available_features)

In [ ]:
def predict_tsunami_from_earthquake(magnitude, depth, latitude, longitude, model=None, scaler=None, feature_columns=None):
    if model is None:
        model = best_model
    if scaler is None:
        scaler = globals().get('scaler')
    if feature_columns is None:
        feature_columns = available_features
    
    earthquake_data = pd.DataFrame({
        'magnitude': [magnitude],
        'depth': [depth],
        'latitude': [latitude],
        'longitude': [longitude],
        'id': ['prediction'],
        'place': ['Unknown'],
        'time': [datetime.now()]
    })
    
    try:
        risk_result = assess_tsunami_risk(earthquake_data, model, scaler, feature_columns)
        
        if len(risk_result) > 0:
            prediction = {
                'earthquake': {
                    'magnitude': magnitude,
                    'depth': depth,
                    'latitude': latitude,
                    'longitude': longitude
                },
                'tsunami_prediction': bool(risk_result.iloc[0]['tsunami_prediction']),
                'tsunami_probability': float(risk_result.iloc[0]['tsunami_probability']),
                'risk_level': risk_result.iloc[0]['risk_level'],
                'is_oceanic': bool(is_oceanic(latitude, longitude)),
                'risk_zone': assign_risk_zone(latitude, longitude),
                'timestamp': datetime.now().isoformat()
            }
        else:
            prediction = {
                'error': 'Could not process earthquake data',
                'timestamp': datetime.now().isoformat()
            }
            
    except Exception as e:
        prediction = {
            'error': f'Prediction failed: {str(e)}',
            'timestamp': datetime.now().isoformat()
        }
    
    return prediction

test_cases = [
    {"name": "Major shallow oceanic earthquake", "mag": 7.5, "depth": 25, "lat": 38.0, "lon": 142.0},
    {"name": "Moderate deep continental earthquake", "mag": 6.2, "depth": 150, "lat": 40.0, "lon": -120.0},
    {"name": "Great shallow Pacific earthquake", "mag": 8.2, "depth": 15, "lat": -15.0, "lon": -175.0},
    {"name": "Small shallow earthquake", "mag": 5.0, "depth": 10, "lat": 35.0, "lon": 25.0}
]

for test in test_cases:
    result = predict_tsunami_from_earthquake(test['mag'], test['depth'], test['lat'], test['lon'])

In [ ]:
import os
os.makedirs('models', exist_ok=True)

model_components = {
    'model': best_model,
    'scaler': scaler,
    'feature_columns': available_features,
    'label_encoders': label_encoders,
    'model_name': best_model_name,
    'performance_metrics': {
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    },
    'training_date': datetime.now().isoformat(),
    'training_data_shape': df_clean.shape
}

with open('models/tsunami_predictor_model.pkl', 'wb') as f:
    pickle.dump(model_components, f)

def generate_tsunami_alert(prediction_result):
    eq = prediction_result['earthquake']
    
    if prediction_result['tsunami_prediction']:
        severity = prediction_result['risk_level'].upper()
        print(f"\n🚨 TSUNAMI ALERT - {severity} RISK 🚨")
        print("="*50)
        print(f"📍 Location: {eq['latitude']:.2f}°, {eq['longitude']:.2f}°")
        print(f"📊 Magnitude: {eq['magnitude']}")
        print(f"🔽 Depth: {eq['depth']} km")
        print(f"🌊 Probability: {prediction_result['tsunami_probability']:.1%}")
        print(f"⚠️ Risk Level: {prediction_result['risk_level']}")
        print(f"🌍 Zone: {prediction_result['risk_zone']}")
        print(f"🌊 Oceanic: {'Yes' if prediction_result['is_oceanic'] else 'No'}")
        print("⚠️ TAKE IMMEDIATE PROTECTIVE ACTIONS ⚠️")
        print("="*50)
    else:
        print(f"\nℹ️ Low Risk Earthquake - Magnitude {eq['magnitude']}")
        print(f"🌊 Tsunami Probability: {prediction_result['tsunami_probability']:.1%}")

test_prediction = predict_tsunami_from_earthquake(7.8, 20, 38.0, 142.0)
generate_tsunami_alert(test_prediction)